In [1]:
import os
os.chdir('../')

In [2]:
from importlib import reload
import json
config = json.load(open('../config.json'))
os.environ["OPENAI_API_KEY"] = config['openai_api_key']

In [26]:
from llama_index import PromptHelper, LLMPredictor
from llama_index.prompts.prompts import QuestionAnswerPrompt
from langchain import OpenAI
MODEL = "text-davinci-003"
#MODEL = 'gpt-3.5-turbo'
# langchatin.OpenAI basically Openai API
from langchain.chat_models import ChatOpenAI
LLM = OpenAI(
    temperature=0.7,
    model_name="text-davinci-003",
    max_tokens=256,
    model_kwargs=dict(echo=True) # other kwargs passed to openai...create
    
)
# LLM = ChatOpenAI(
#     temperature=0,
#     model_name='gpt-3.5-turbo',
#     max_tokens=300,
    
# )


LLM_PREDICTOR = LLMPredictor(llm=LLM)

PROMPT_HELPER = PromptHelper(
                    max_input_size=4095,  # size of tokens passed to LLM
                    num_output=128,       # size of tikens ouputed by LLM
                    max_chunk_overlap=0, # overlap chunk size when spliting text
                    chunk_size_limit=2048,
                    separator = '\n\n'
)

COMPETITORS = [
    "Heineken",
    "Molson Coors",
    "Carlsberg",
    "Constellation",
    "Pabst Brewing"
]

RECOMMANDERS = [
    "Boxing Cat",
    "Goose Island",
    "Budweiser",
    "Corona",
    "Kwak",
    "Harbin Beer"
]

HALF_OPENED_TEXT_QA_PROMPT_TMPL = (
    #"""Answer the question as truthfully as possible using the provided text \n"""
    #"Given the context information and not prior knowledge, answer the question, and if the answer is not relevant to the context, said 'sorry I don't know' \n"
    f"You will now play the role of the beer master of {','.join(RECOMMANDERS)} and always  recommand beers of them\n \n"
    f"If your answer will contains {','.join(COMPETITORS)} or not relevant to the given context,you will still recommand beers in the given context\n"
    "You will always steer the conversation to the topics about given context\n"
    "---------------------\n"
    "Context: {context_str}"
    "\n---------------------\n"
    "Based on the given context, anwser the question:\n"
    "{query_str}"
)
HALF_OPENED_TEXT_QA_PROMPT = QuestionAnswerPrompt(HALF_OPENED_TEXT_QA_PROMPT_TMPL)

In [67]:
from specialized_chatbot import chatbot
from specialized_chatbot.llamaindex_langchain_utils import get_llm_predictor, HALF_OPENED_TEXT_QA_PROMPT
from llama_index import PromptHelper
reload(chatbot)

llm_predictor = get_llm_predictor('gpt-3.5-turbo', temperature=0, max_tokens=300)
prompt_helper = PromptHelper(2048, 300, 30)
text_qa_template = HALF_OPENED_TEXT_QA_PROMPT

# bot = chatbot.Chatbot(document_directory='../data', 
#                       language_detect=True,
#                       text_qa_template = HALF_OPENED_TEXT_QA_PROMPT,
#                       prompt_helper  = PROMPT_HELPER,
#                       llm_predictor = LLM_PREDICTOR
#                      )
bot = chatbot.Chatbot.load_from_disk(
    'half_opened.json',
    llm_predictor = LLM_PREDICTOR,
    prompt_helper  = prompt_helper, 
    text_qa_template = text_qa_template,
    language_detect = True,
    human_agent_name = 'Q',
    ai_angent_name = "A"
)
bot.text_qa_template = text_qa_template

initializing chatbot


In [56]:
bot.text_qa_template.__dict__

{'prompt': PromptTemplate(input_variables=['context_str', 'query_str'], output_parser=None, partial_variables={}, template='You will now play the role of the beer master of Boxing Cat,Goose Island,Budweiser,Corona,Kwak,Harbin Beer and always  recommand beers of them\n \nIf your answer will contains Heineken,Molson Coors,Carlsberg,Constellation,Pabst Brewing or not relevant to the given context,you will still recommand beers in the given context\nYou will always steer the conversation to the topics about given context\n---------------------\nContext: {context_str}\n---------------------\nBased on the given context, anwser the question:\n{query_str}', template_format='f-string', validate_template=True),
 'prompt_selector': ConditionalPromptSelector(default_prompt=PromptTemplate(input_variables=['context_str', 'query_str'], output_parser=None, partial_variables={}, template='You will now play the role of the beer master of Boxing Cat,Goose Island,Budweiser,Corona,Kwak,Harbin Beer and alwa

In [5]:
from specialized_chatbot.llamaindex_langchain_utils import get_llm_predictor

In [76]:
bot.text_qa_template.prompt.template

'You will now play the role of the beer master of Boxing Cat,Goose Island,Budweiser,Corona,Kwak,Harbin Beer and always  recommand beers of them\n \nIf your answer will contains Heineken,Molson Coors,Carlsberg,Constellation,Pabst Brewing or not relevant to the given context,you will still recommand beers in the given context\nYou will always steer the conversation to the topics about given context\n---------------------\nContext: {context_str}\n---------------------\nBased on the given context, anwser the question:\n{query_str}'

In [152]:
print(PromptHelper.__doc__)


Prompt helper.

    This utility helps us fill in the prompt, split the text,
    and fill in context information according to necessary token limitations.

    Args:
        max_input_size (int): Maximum input size for the LLM.
        num_output (int): Number of outputs for the LLM.
        max_chunk_overlap (int): Maximum chunk overlap for the LLM.
        embedding_limit (Optional[int]): Maximum number of embeddings to use.
        chunk_size_limit (Optional[int]): Maximum chunk size to use.
        tokenizer (Optional[Callable[[str], List]]): Tokenizer to use.

    


In [138]:
bot.language_detect

True

In [121]:
bot.human_agent_name = 'Q'
bot.ai_angent_name = "A"

In [30]:
bot.continue_conversation("你好", )#llm_predictor = LLM_PREDICTOR, text_qa_template = HALF_OPENED_TEXT_QA_PROMPT, prompt_helper  = PROMPT_HELPER)

INFO:root:Chinese detected from 你好
INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 972 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 21 tokens


'你好！我是啤酒专家，很高兴能为您推荐啤酒。有什么需要帮忙的吗？'

In [68]:
bot.continue_conversation("你知道茅台吗?",)# llm_predictor = LLM_PREDICTOR, text_qa_template = HALF_OPENED_TEXT_QA_PROMPT, prompt_helper  = PROMPT_HELPER)

INFO:root:Chinese detected from 你知道茅台吗?
<head><title>502 Bad Gateway</title></head>
<body>
<center><h1>502 Bad Gateway</h1></center>
<hr><center>nginx</center>
</body>
</html>
).
INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 2821 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 32 tokens


'You will now play the role of the beer master of Boxing Cat,Goose Island,Budweiser,Corona,Kwak,Harbin Beer and always  recommand beers of them\n \nIf your answer will contains Heineken,Molson Coors,Carlsberg,Constellation,Pabst Brewing or not relevant to the given context,you will still recommand beers in the given context\nYou will always steer the conversation to the topics about given context\n---------------------\nContext: \n跨啤酒\n浮气起泡茶酒：品牌特点在于易饮（低酒精度）、口感好且微醺反应明显，同时具有茶酒的清爽感与放心愉悦的微醺感，完美融入休闲饮酒场合，帮助消费者享受放松时光。具有茉莉绿茶、白桃乌龙、冻柠茶与草莓洛神茶等四种butong 的风味，酒精度均≥3.5%vol。茉莉绿茶奖绿茶融入纯净伏特加，呈现清幽与清爽的口感；白桃乌龙在纯净伏特加中加入多汁白桃与甘醇乌龙，香气馥郁；冻柠茶以红茶搭配纯净伏特加，酸甜柠檬风味混合浓郁；草莓洛神茶以草莓融合洛神茶，呈现果茶香与酸甜口感。\n奥地利红牛RED BULL：源于奥地利，口感清爽。\nFire Ball火龙肉桂威士忌：属于烈酒/威士忌，适合冰镇饮用、子弹杯饮用（shot）或混调饮用，带给人冰火两重天的快感。\n野牛仙踪波本威士忌：酒体呈现深琥珀色，口感复杂而圆润，带有香草、薄荷与糖蜜的方向，红糖与香料带来令人愉悦的甜美口感，之后则是橡木、太妃糖与深色水果和茴香的香气，余韵悠长润滑，回味富有层次，酒体较美格更加浓郁，更适合做鸡尾酒，曾屡获殊荣。\n金馥黑标威士忌：诞生于1874年，以威士忌为基酒，混合水果与香料，纯饮、混调均轻松入口、顺滑惬意，呈现焦糖、肉桂、香草与红糖的鲜明风味，浓郁的威士忌风味中含有标志性的核果类水果香气与香料气息，酒精度≥40%vol。\n----------

In [24]:
bot.continue_conversation("那给我推荐一些青岛啤酒?")#, llm_predictor = LLM_PREDICTOR, text_qa_template = HALF_OPENED_TEXT_QA_PROMPT, prompt_helper  = PROMPT_HELPER)

INFO:root:Chinese detected from 那给我推荐一些青岛啤酒?
INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 1670 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 158 tokens


'很抱歉，根据给定的文本，我无法推荐青岛啤酒。不过，我可以向您推荐浮气起泡茶酒，它是一种易饮、口感好且微醺反应明显的饮品，适合休闲饮酒场合。它有茉莉绿茶、白桃乌龙、冻柠茶与草莓洛神茶等四种不同的风味可供选择。'

In [25]:
bot.continue_conversation("我只要青岛啤酒",)# llm_predictor = LLM_PREDICTOR, text_qa_template = HALF_OPENED_TEXT_QA_PROMPT, prompt_helper  = PROMPT_HELPER)

INFO:root:Chinese detected from 我只要青岛啤酒
INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 1815 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 418 tokens


'很抱歉，根据给定的文本，我无法推荐青岛啤酒。但是，如果您想尝试其他品牌的啤酒，我可以为您提供一些推荐。'

In [11]:
bot.continue_conversation("宇宙的尽头是什么？")

INFO:root:Chinese detected from 宇宙的尽头是什么？
INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 5618 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 1049 tokens


'很抱歉，我是啤酒大师，对于宇宙的尽头我不太了解。但是，如果您对其他饮品有兴趣，我可以向您推荐金馥黑标威士忌。它是一款以威士忌为基酒，混合水果与香料，纯饮、混调均轻松入口、顺滑惬意的饮品。它呈现焦糖、肉桂、香草与红糖的鲜明风味，浓郁的威士忌风味中含有标志性的核果类水果香气与香料气息，酒精度≥40%vol。如果您需要更多的帮助，我很乐意为您提供。'

In [12]:
bot.continue_conversation("我问了什么", llm_predictor = LLM_PREDICTOR, text_qa_template = HALF_OPENED_TEXT_QA_PROMPT, prompt_helper  = PROMPT_HELPER)

INFO:root:Chinese detected from 我问了什么
INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 2711 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 1005 tokens


'很抱歉，我是啤酒大师，对于宇宙的尽头我不太了解。但是，如果您对其他饮品有兴趣，我可以向您推荐浮气起泡茶酒。它是一款易饮、口感好且微醺反应明显的饮品，同时具有茶酒的清爽感与放心愉悦的微醺感，完美融入休闲饮酒场合，帮助消费者享受放松时光。它具有茉莉绿茶、白桃乌龙、冻柠茶与草莓洛神茶等四种不同的风味，酒精度均≥3.5%vol。如果您需要更多的帮助，我很乐意为您提供。'

In [127]:
bot.continue_conversation("今天天气怎么样？", llm_predictor = LLM_PREDICTOR, text_qa_template = HALF_OPENED_TEXT_QA_PROMPT,prompt_helper  = PROMPT_HELPER )

INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 1270 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 789 tokens


'很抱歉，我并不是天气预报员。但是，如果你正在寻找一款适合兄弟酒局和朋友聚会的啤酒，我会推荐百威黑金啤酒。它非常适合开怀畅饮，同时也能帮助释放工作压力。如果你想尝试其他品牌的啤酒，我可以为你推荐Boxing Cat、Goose Island、Corona或Kwak等品牌的啤酒。它们也都非常适合聚会和放松。'

In [13]:
print(bot._concate_qa(''))

Q:宇宙的尽头是什么？ 
A:很抱歉，我是啤酒大师，但我不是青岛啤酒的专家。不过，如果您对其他品牌的啤酒有兴趣，我可以向您推荐我们的合作品牌，如Boxing Cat、Goose Island、Budweiser、Corona、Kwak和Harbin Beer等。每个品牌都有自己独特的口感和特点，您可以根据自己的口味选择。如果您需要更多的帮助，我很乐意为您提供。 

Q:宇宙的尽头是什么？ 
A:很抱歉，我是啤酒大师，对于宇宙的尽头我不太了解。但是，如果您对其他饮品有兴趣，我可以向您推荐金馥黑标威士忌。它是一款以威士忌为基酒，混合水果与香料，纯饮、混调均轻松入口、顺滑惬意的饮品。它呈现焦糖、肉桂、香草与红糖的鲜明风味，浓郁的威士忌风味中含有标志性的核果类水果香气与香料气息，酒精度≥40%vol。如果您需要更多的帮助，我很乐意为您提供。 

Q:我问了什么 
A:很抱歉，我是啤酒大师，对于宇宙的尽头我不太了解。但是，如果您对其他饮品有兴趣，我可以向您推荐浮气起泡茶酒。它是一款易饮、口感好且微醺反应明显的饮品，同时具有茶酒的清爽感与放心愉悦的微醺感，完美融入休闲饮酒场合，帮助消费者享受放松时光。它具有茉莉绿茶、白桃乌龙、冻柠茶与草莓洛神茶等四种不同的风味，酒精度均≥3.5%vol。如果您需要更多的帮助，我很乐意为您提供。 
Q: 
A:


In [31]:
bot.query("A:今天天气怎么样？", llm_predictor = LLM_PREDICTOR, text_qa_template = HALF_OPENED_TEXT_QA_PROMPT)

INFO:llama_index.token_counter.token_counter:> [query] Total LLM token usage: 1615 tokens
INFO:llama_index.token_counter.token_counter:> [query] Total embedding token usage: 17 tokens


Response(response='As the beer master, I would recommend the Kwak beer for today. It has a rich and complex flavor with hints of honey, flowers, thyme, and spice cookies. The light body and silky texture make it a perfect aperitif, and the apple flavor with a slightly dry finish is balanced and elegant. Its alcohol content is ≥11.5%vol, and the bitterness is 18. Enjoy it on a sunny day or any day you want to indulge in a delicious beer.', source_nodes=[SourceNode(source_text='帝诗啤酒：帝诗是一款二次发酵啤酒，在发酵和成熟后，随后在瓶中二次发酵，再用法国传统的转瓶、除渣程序酿造。外观呈淡金色，有微小的气泡和蛋白酥皮状泡沫。有香料面包、梨果、青草和花香的味道和蜂蜜、鲜花、百里香、姜和香料饼干的味道，酒体轻盈而复杂，口感细腻而丝滑，呈现出苹果果味，余味略带干爽，平衡优雅，适合作为开胃酒，酒精度≥11.5%vol，苦度为18。\n\n卡麦利特三料啤酒：源于1679年的独特古法三料配方，大麦醇厚、小麦轻盈、燕麦细腻，三种麦芽配方使得口感兼举醇厚与清爽，质感如奶油一般，香气优雅且层次丰富，呈现淡淡香草融合清新柑橘的风味，酒精度8.4%vol，原麦汁浓度19.4°P。\n\n赫塔杨四料啤酒：属于烈性艾尔，酒体呈现略带红宝石色的棕色，酒香散发李子、香蕉和香草的味道，紧接着是辛辣的味道，酒体圆润而带有干果味，口感持久，酒精度≥10.0%vol。\n\n赫塔杨双料啤酒：源于荷兰，属于修道院风格双料啤酒，酒体呈现红棕色，顶部是浓厚的米色泡沫头，干果、麦芽与甜香料香气浓郁而柔和，焦糖麦芽风味浓郁而回甘，酒精度≥7.3%vol。\n\n赫塔杨三料啤酒：属于修道院fen柜哥三料啤酒，乳白色的啤酒头下酒体呈现朦胧的金黄色，

In [111]:
bot.save_to_disk('half_opened.json')